# Reading 12: Part 3 - Finite State Machines

Sometimes, using a direct set of <i>input to output</i> signals is insufficient to solve the problem. Consider the case of a vending machine. Vending machine use small <a href = "https://en.wikipedia.org/wiki/Embedded_system">embedded systems</a> to keep track of how much money you have dispensed, and when to vend specific items. For example, if we have paid <b>1.00</b> and we enter a quarter, there are different results if we are purchasing an item that is <b>1.75</b>, <b>1.25</b>, or <b>1.00</b>. Coding the result for each individual item takes up a lot of space on the limited embedded machine. We can do better.


We can write digital logic that acts like a graph called a <b>Finite State Machine</b>. A <b>Finite State Machine</b> consists of:<br />
<ol>
    <li>A set of states (represented as nodes)</li>
    <li>An initial state</li>
    <li>A set of transitions between states (represented by edges)</li>
    <li>A set of control input signals</li>
</ol>

An example of a simple mechanism that can be modeled by a state machine is a turnstile. A turnstile, used to control access to subways and amusement park rides, is a gate with three rotating arms at waist height, one across the entryway. Initially the arms are locked, blocking the entry, preventing patrons from passing through. Depositing a coin or token in a slot on the turnstile unlocks the arms, allowing a single customer to push through. After the customer passes through, the arms are locked again until another coin is inserted.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Turnstile_state_machine_colored.svg/330px-Turnstile_state_machine_colored.svg.png"></center>

The turnstile has two possible states: <b>Locked</b> and <b>Unlocked</b>. There are two possible inputs that affect its state: putting a coin in the slot (<b>coin</b>) and pushing the arm (<b>push</b>). In the locked state, pushing on the arm has no effect; no matter how many times the input <b>push</b> is given, it stays in the locked state. Putting a <b>coin</b> in <i>shifts the state</i> from Locked to Unlocked. In the unlocked state, putting additional coins in has no effect; that is, giving additional coin inputs does not change the state. However, a customer pushing through the arms, giving a push input, shifts the state back to Locked.


Consider the image below: The format is <code>state_name / output</code>. For example, any time you reach the state <code>q0</code>, the output will be a <code>0</code>. Conversely, any time you reach the state <code>q3</code>, the output will be a <code>0</code>.

<b>Start State</b>: <code>q0</code><br />
<b>Control Signals</b>: <code>{1, 1, 0, 1, 0, 0, 0, 1, 1}</code><br />

Here is how you would derive the results:
<ol>
    <li>The start state for this step is <code>q0</code>, so the output is <code>0</code> and the control input is <code>1</code>. The edge takes to <font color="red"><code>q2</code></font></li>
    <li>The start state for this step is <code>q2</code>, so the output is <code>0</code>, and the control input is <code>1</code>. The edge takes to <code>q2</code></li>
    <li>The start state for this step is <code>q2</code>, so the output is <code>0</code>, and the control input is <code>0</code>. The edge takes to <code>q4</code></li>
    <li>The start state for this step is <code>q4</code>, so the output is <code>1</code>, and the control input is <code>1</code>. The edge takes to <code>q3</code></li>
    <li>The start state for this step is <code>q3</code>, so the output is <code>1</code>, and the control input is <code>0</code>. The edge takes to <code>q4</code></li>
    <li>The start state for this step is <code>q4</code>, so the output is <code>1</code>, and the control input is <code>0</code>. The edge takes to <code>q1</code></li>
    <li>The start state for this step is <code>q1</code>, so the output is <code>0</code>, and the control input is <code>0</code>. The edge takes to <code>q1</code></li>
    <li>The start state for this step is <code>q1</code>, so the output is <code>0</code>, and the control input is <code>1</code>. The edge takes to <code>q3</code></li>
        <li>The start state for this step is <code>q3</code>, so the output is <code>1</code>, and the control input is <code>1</code>. The edge takes to <code>q3</code></li>
</ol>


<img src = "https://media.geeksforgeeks.org/wp-content/uploads/1-43.jpg" height=500 width=500>

The solution is <code>{0, 0, 0, 1, 1, 1, 0, 0, 1}</code>

Solution Table:

| Start State | Output | Control | Next State |
|---|---|---|---|
|q0|<font color="red">0</font>|1|q2|
|q2|<font color="red">0</font>|1|q2|
|q2|<font color="red">0</font>|0|q4|
|q4|<font color="red">1</font>|1|q3|
|q3|<font color="red">1</font>|0|q4|
|q4|<font color="red">1</font>|0|q1|
|q1|<font color="red">0</font>|0|q1|
|q1|<font color="red">0</font>|1|q3|
|q3|<font color="red">1</font>|1|q2|

In [1]:
# Like in Part 1, we will start by imporing pyrtl
from pyrtl import *
import pyrtl

### Now, we can define states. 

Using the same nodes and transitions as above, hHere are the outputs we will create for our new FSM:

<ol>
    <li><b>State Name:</b><code>q0</code>, <b>Outputs:</b> <code>out = 0</code></li>
    <li><b>State Name:</b><code>q1</code>, <b>Outputs:</b> <code>out = 0</code></li>
    <li><b>State Name:</b><code>q2</code>, <b>Outputs:</b> <code>out = 0</code></li>
    <li><b>State Name:</b><code>q3</code>, <b>Outputs:</b> <code>out = 1</code></li>
    <li><b>State Name:</b><code>q4</code>, <b>Outputs:</b> <code>out = 1</code></li>
</ol>

### Now that we have our functions and states defined, we can define our transitions

The difference is that we can't use the <code>if/else</code> statements in the Finite State Machine since we are performing Digital Logic.<br /> So we use <code>with</code> statements, which in PyRTL are equivalent to <code>elif</code>

### First, define each state in the FSM

To promote modularity, and reduce the challenges of troubleshooting, keep the "current state operations" and the "next state calculations separate.

In [2]:
# For this function, we assume the initial state has already been executed impemented

def curr_state_op( curr_state ):
    
    # In-Class: Must include with pyrtl.conditional_assignment
    with pyrtl.conditional_assignment:
        
        # In-Class: Create two intermediate WireVectors of bitwidth 1
        state_out = pyrtl.WireVector(1)
        
        # In-Class: Perform the transition
        
        # State q0
        with curr_state == 0:
            state_out |= 0
        
        # State q1
        with curr_state == 1:
            state_out |= 0
        
        # State q2
        with curr_state == 2:
            state_out |= 0

        # State q3
        with curr_state == 3:
            state_out |= 1

        # State q4
        with curr_state == 4:
            state_out |= 1
            
        # Return both wire outputs
        return state_out

In [3]:
# For this function, we assume the initial state has already been executed impemented

def next_state( curr_state, control_signal ):
    
    # Every time you perform a conditional assignment
    with pyrtl.conditional_assignment:
            
        # Perform the transition
        
        # State q0
        with curr_state == 0:
            
            with control_signal == 0:
                curr_state.next |= 1
                
            with control_signal == 1:
                curr_state.next |= 2
                
        # State q1
        with curr_state == 1:
            
            with control_signal == 0:
                curr_state.next |= 1
                
            with control_signal == 1:
                curr_state.next |= 3
                
        # State q2
        with curr_state == 2:
            
            with control_signal == 0:
                curr_state.next |= 4
                
            with control_signal == 1:
                curr_state.next |= 2
                
        # State q3
        with curr_state == 3:
            
            with control_signal == 0:
                curr_state.next |= 4
                
            with control_signal == 1:
                curr_state.next |= 2
                

        # State q4
        with curr_state == 4:
            
            with control_signal == 0:
                curr_state.next |= 1
                
            with control_signal == 1:
                curr_state.next |= 3
        
        # Return the updated memory
        return curr_state.next

### Finally, we can perform the operations for the FSM

In [4]:
def example_fsm_simulate( ):
    
    # Step 1 - Reset the working block
    pyrtl.reset_working_block()
    
    # Step 2 - Create the input and ouput wires
    control_signal = pyrtl.Input(2, 'control_signal')
    output = pyrtl.Output(1, 'out')
    
    curr_state = pyrtl.Register(5, 'curr_state')
    
    # Step 3 - Save to an intermediate value using example_fsm the function
    inter_output = curr_state_op( curr_state )
    curr_state = next_state( curr_state, control_signal )
    
    # Step 3b - Connect them to the outputs using the <<= operator
    output <<= inter_output
    
    # Step 4 - Start the design simulation
    sim = pyrtl.Simulation()
    
    # Step 5 - Create lists for the inputs
    control_signals = [1,1,0,1,0,0,0,1]
    
    
    # Step 6 - Loop through and simuluate
    for value in range(len(control_signals)):

        sim.step({
            
            # Note the control signal is one bit shorter since we need the first three for q0 state
            'control_signal' : control_signals[value],
        })
    
    # Step 7- Render the trace
    sim.tracer.render_trace()

In [5]:
example_fsm_simulate()

<IPython.core.display.Javascript object>